In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import pandas as pd
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import time
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
def get_device():
    if torch.cuda.is_available():
        # Get the number of available GPUs
        n_gpu = torch.cuda.device_count()
        print(f"Number of GPUs available: {n_gpu}")
        
        if n_gpu >= 3:  # If GPU 2 (index 2) is available
            print("Using GPU 2")
            return torch.device('cuda:2')
        elif n_gpu > 0:  # If any GPU is available
            print(f"GPU 2 not available, using GPU 0")
            return torch.device('cuda:0')
    
    print("No GPU available, using CPU")
    return torch.device('cpu')

class RealEstateDataset(Dataset):
    def __init__(self, data):
        """
        Custom dataset for real estate data with temporal sequence handling
        """
        # Static features remain the same
        self.static_features = torch.FloatTensor(
            data[['latitude', 'longitude', 'neighbourhood_cleansed_encoded']].values
        )
        
        # Temporal features - organize into groups
        temporal_cols = [
            # Time features
            'DTF_day_of_week', 'DTF_month', 'DTF_is_weekend',
            'DTF_season_sin', 'DTF_season_cos',
            # Price history features
            'price_lag_90d', 'price_lag_120d', 'price_lag_150d', 'price_lag_180d',
            # Rolling window features - 30 days
            'rolling_mean_30d', 'rolling_std_30d', 'rolling_max_30d', 'rolling_min_30d',
            # Rolling window features - 60 days
            'rolling_mean_60d', 'rolling_std_60d', 'rolling_max_60d', 'rolling_min_60d',
            # Rolling window features - 90 days
            'rolling_mean_90d', 'rolling_std_90d', 'rolling_max_90d', 'rolling_min_90d'
        ]
        
        # Get temporal data
        temp_data = data[temporal_cols].values
        
        # Instead of reshaping, we'll keep the temporal features flat for now
        self.temporal_features = torch.FloatTensor(temp_data)
        
        # Target
        self.price = torch.FloatTensor(data['price'].values)
        
    def __len__(self):
        return len(self.price)
    
    def __getitem__(self, idx):
        return {
            'static': self.static_features[idx],
            'temporal': self.temporal_features[idx],
            'price': self.price[idx]
        }

class RealEstateNN(nn.Module):
    def __init__(self, feature_dims):
        """
        Neural network model with GRU for temporal features
        """
        super(RealEstateNN, self).__init__()
        
        # Static features branch remains similar
        self.static_branch = nn.Sequential(
            nn.Linear(feature_dims['static'], 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        
        # Temporal features processing
        self.temporal_branch = nn.Sequential(
            nn.Linear(feature_dims['temporal'], 128),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU()
        )
        
        # Combined layers
        combined_dim = 32 + 32  # Static + Temporal
        self.combined_layers = nn.Sequential(
            nn.Linear(combined_dim, 64),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1)
        )
        
    def forward(self, static, temporal):
        # Process static features
        static_out = self.static_branch(static)
        
        # Process temporal features
        temporal_out = self.temporal_branch(temporal)
        
        # Combine branches
        combined = torch.cat([static_out, temporal_out], dim=1)
        
        # Final prediction
        return self.combined_layers(combined)

class RealEstateTrainer:
    def __init__(self, feature_dims, device='cuda:2'):
        """
        Trainer class for the real estate neural network
        """
        self.device = torch.device(device)
        self.model = RealEstateNN(feature_dims).to(self.device)
        self.criterion = nn.MSELoss()
        self.optimizer = optim.Adam(self.model.parameters(), lr=0.001)
        
    def train_epoch(self, train_loader):
        """
        Train for one epoch
        """
        self.model.train()
        total_loss = 0
        
        for batch in train_loader:
            # Move batch to GPU
            static = batch['static'].to(self.device)
            temporal = batch['temporal'].to(self.device)
            price = batch['price'].to(self.device)
            
            # Forward pass
            self.optimizer.zero_grad()
            output = self.model(static, temporal)
            loss = self.criterion(output.squeeze(), price)
            
            # Backward pass
            loss.backward()
            self.optimizer.step()
            
            total_loss += loss.item()
            
        return total_loss / len(train_loader)
    
    @torch.no_grad()
    def validate(self, val_loader):
        """
        Validate the model
        """
        self.model.eval()
        total_loss = 0
        
        for batch in val_loader:
            # Move batch to GPU
            static = batch['static'].to(self.device)
            temporal = batch['temporal'].to(self.device)
            price = batch['price'].to(self.device)
            
            # Forward pass
            output = self.model(static, temporal)
            loss = self.criterion(output.squeeze(), price)
            
            total_loss += loss.item()
            
        return total_loss / len(val_loader)
    
    @torch.no_grad()
    def predict(self, test_loader):
        """
        Make predictions
        """
        self.model.eval()
        predictions = []
        
        for batch in test_loader:
            # Move batch to GPU
            static = batch['static'].to(self.device)
            temporal = batch['temporal'].to(self.device)
            
            # Forward pass
            output = self.model(static, temporal)
            predictions.append(output.cpu().numpy())
            
        return np.concatenate(predictions).squeeze()

def train_model(train_data, val_data, batch_size=32, epochs=100, device='cuda:2'):
    """
    Main training function with streamlined progress display
    """
    # Create datasets
    train_dataset = RealEstateDataset(train_data)
    val_dataset = RealEstateDataset(val_data)
    
    # Create data loaders
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size)
    
    # Initialize feature dimensions
    feature_dims = {
        'static': 3,  # latitude, longitude, neighbourhood
        'temporal': 21  # all temporal features
    }
    
    # Initialize trainer
    trainer = RealEstateTrainer(feature_dims, device)
    
    # Training loop
    best_val_loss = float('inf')
    patience = 10
    patience_counter = 0
    
    # Progress bar for epochs
    pbar = tqdm(range(epochs), desc='Training')
    
    for epoch in pbar:
        # Training phase
        train_loss = trainer.train_epoch(train_loader)
        
        # Validation phase
        val_loss = trainer.validate(val_loader)
        
        # Update progress bar
        pbar.set_postfix({
            'train_loss': f'{train_loss:.4f}',
            'val_loss': f'{val_loss:.4f}',
            'patience': f'{patience_counter}/{patience}'
        })
        
        # Early stopping check
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            patience_counter = 0
            torch.save(trainer.model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= patience:
                print("\nEarly stopping triggered!")
                break
    
    return trainer

In [8]:
# Load preprocessed data
print("Loading preprocessed data...")
train_data = pd.read_csv(r'C:\Users\mvk\Documents\DATA_school\thesis\train_data_2024.csv')
test_data = pd.read_csv(r'C:\Users\mvk\Documents\DATA_school\thesis\test_data_2025.csv')
print("Data loaded successfully.")

# Reduce the number of listings to 500 for testing and development
train_data = train_data.sample(n=2000, random_state=42)
test_data = test_data.sample(n=2000, random_state=42)
print("Reduced the number of listings to 2000 for testing and development.")

Loading preprocessed data...
Data loaded successfully.
Reduced the number of listings to 2000 for testing and development.


In [9]:
# Get the appropriate device
device = get_device()

# Train model
print("\nStarting model training...")
trainer = train_model(train_data, test_data, device=device)

# Evaluate on test set
print("\n=== Evaluating Model ===")
print("Creating test dataset and loader...")
test_dataset = RealEstateDataset(test_data)
test_loader = DataLoader(test_dataset, batch_size=32)

print("Making predictions...")
predictions = trainer.predict(test_loader)

# Calculate metrics
true_prices = test_data['price'].values
metrics = {
    'rmse': np.sqrt(mean_squared_error(true_prices, predictions)),
    'mae': mean_absolute_error(true_prices, predictions),
    'r2': r2_score(true_prices, predictions),
    'mape': np.mean(np.abs((true_prices - predictions) / true_prices)) * 100
}

print("\n=== Final Model Performance ===")
for metric, value in metrics.items():
    print(f"{metric.upper()}: {value:.4f}")

Number of GPUs available: 1
GPU 2 not available, using GPU 0

Starting model training...


Training:  20%|██        | 20/100 [00:04<00:19,  4.10it/s, train_loss=0.0322, val_loss=0.0120, patience=9/10]


Early stopping triggered!

=== Evaluating Model ===
Creating test dataset and loader...
Making predictions...

=== Final Model Performance ===
RMSE: 0.1099
MAE: 0.0700
R2: 0.9902
MAPE: 20.7810
